# Data Processing

Change the data_path to your directory as needed

1.   key: the key pressed
2.   response_id: unique identifying code for this participant typing this
target sentence
3. response_content: response for this sentence as typed in full
4. participant_id: unique ID for participant
5. sentence_id: identifier for target sentence
6. sentence_content: target sentence as displayed to the participant
7. diagnosis:
> *   1=Typist has had a diagnosis of Parkinson's disease;
> * 0=Typist has not had a diagnosis of Parkinson's disease
8. keydown: Timestamp for press of key in milliseconds
9. keyup: Timestamp for release of key in milliseconds

In [1]:
import pandas as pd
import numpy as np
import os
import re
import json

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#add path to data
data_path = '/content/drive/My Drive/266 Assignments/266 Final Project'

files = os.listdir(data_path)
files = [x for x in files if '.csv' in x]
files

['online_english_fold_all.csv',
 'MedicationInfo.csv',
 'CoNLL_2020_Online_English.csv',
 'cleaned_data.csv',
 'depreciated_wrongAUC_calc_metrics.csv',
 'metrics.csv',
 'analyzed_metrics.csv']

## Data processing
*   Create key_duration_ms column, which is keyup - keydown
*   Replace non-character keys (e.g. shift, enter, space, backspace, control) with special tokens

In [3]:
df = pd.read_csv(os.path.join(data_path, files[files.index('CoNLL_2020_Online_English.csv')]))
df['key_duration_ms'] = df['keyup'] - df['keydown']
df

,key,response_id,response_content,participant_id,sentence_id,sentence_content,diagnosis,keydown,keyup,key_duration_ms
0,shift,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80077.6719,80672.2719,594.600
1,b,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80511.5719,80615.2719,103.700
2,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80800.2719,80861.4719,61.200
3,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80949.9719,81076.3719,126.400
4,s,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,81305.5719,81480.7719,175.200
...,...,...,...,...,...,...,...,...,...,...
437910,r,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709861.2300,709965.3350,104.105
437911,k,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709997.2450,710077.2900,80.045
437912,e,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710165.2550,710309.3200,144.065
437913,t,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710413.2600,710509.4550,96.195


# Calculate flight time

In [6]:
df['response_id'].unique()

array([  39,   37,   41, ..., 2427, 2428, 2429])

In [13]:
responses = df['response_id'].unique()

new_data = []

for i in responses:
    temp_df = df[df['response_id'] == i]
    temp_df['flight_time'] = temp_df['keydown'].shift(-1) - temp_df['keydown']
    temp_df.at[temp_df.index[-1], 'flight_time'] = pd.NA
    new_data.append(temp_df)

df = pd.concat(new_data)

df

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['flight_time'] = temp_df['keydown'].shift(-1) - temp_df['keydown']
<ipython-input-13-3090e448a3a9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['flight_time'] = temp_df['keydown'].shift(-1) - temp_df['keydown']
<ipython-input-13-3090e448a3a9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[

,key,response_id,response_content,participant_id,sentence_id,sentence_content,diagnosis,keydown,keyup,key_duration_ms,flight_time
0,shift,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80077.6719,80672.2719,594.600,433.900
1,b,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80511.5719,80615.2719,103.700,288.700
2,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80800.2719,80861.4719,61.200,149.700
3,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80949.9719,81076.3719,126.400,355.600
4,s,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,81305.5719,81480.7719,175.200,560.900
...,...,...,...,...,...,...,...,...,...,...,...
437910,r,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709861.2300,709965.3350,104.105,136.015
437911,k,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709997.2450,710077.2900,80.045,168.010
437912,e,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710165.2550,710309.3200,144.065,248.005
437913,t,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710413.2600,710509.4550,96.195,831.980


In [ ]:
# #check distribution of key times per one response
# df.loc[df['response_id']==53, 'key_duration_ms'].plot.hist()

In [14]:
key_vocab = df['key'].unique()

#regex pattern
# pattern = r"^[\w,!.@#$%^}\&_{='/:);(|[\]]$"
pattern = r"^[\w,!.@#$%^}\&_{='/:);(|[\]\"'\\\\-]$"

#include only alphanumeric characters and special characters
vocab_subset = []

for i in key_vocab:
  matches = re.findall(pattern, i)
  vocab_subset.extend(matches)

vocab_subset.sort()

In [15]:
# Convert and write JSON object to file
file_path = os.path.join(data_path, "character_map.json")
with open(file_path, "w") as outfile:
    json.dump(vocab_subset, outfile)

In [16]:
remaining_vocab = list(set(key_vocab) - set(vocab_subset))
remaining_vocab

['shift',
 '\n',
 'alt',
 'mediaprevioustrack',
 'arrowright',
 'delete',
 'meta',
 'spacebar',
 ' ',
 'arrowdown',
 'f11',
 'pagedown',
 'process',
 'insert',
 'end',
 'contextmenu',
 'home',
 'arrowup',
 'f16',
 'numlock',
 'unidentified',
 'enter',
 'arrowleft',
 'control',
 'tab',
 'backspace',
 'capslock']

In [17]:
#both ' ' and 'spacebar' are classed as '[SPCE]'
df.loc[df['key'] == ' ', 'key'] = 'spacebar'

#get all punctuation and alphanumeric characters
def clean_key(key):
    if key in vocab_subset:
        return key
    elif key in remaining_vocab:
        return '[' + key +']'
    else:
        return None
df['key_clean'] = df['key'].apply(clean_key)


df

,key,response_id,response_content,participant_id,sentence_id,sentence_content,diagnosis,keydown,keyup,key_duration_ms,flight_time,key_clean
0,shift,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80077.6719,80672.2719,594.600,433.900,[shift]
1,b,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80511.5719,80615.2719,103.700,288.700,b
2,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80800.2719,80861.4719,61.200,149.700,o
3,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80949.9719,81076.3719,126.400,355.600,o
4,s,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,81305.5719,81480.7719,175.200,560.900,s
...,...,...,...,...,...,...,...,...,...,...,...,...
437910,r,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709861.2300,709965.3350,104.105,136.015,r
437911,k,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709997.2450,710077.2900,80.045,168.010,k
437912,e,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710165.2550,710309.3200,144.065,248.005,e
437913,t,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710413.2600,710509.4550,96.195,831.980,t


In [18]:
clean_vocab = list(df['key_clean'].unique())
#get a list of the special keys
special_tokens = []
for i in clean_vocab:
  if '[' in i and ']' in i:
    special_tokens.append(i)

special_tokens

['[shift]',
 '[backspace]',
 '[spacebar]',
 '[enter]',
 '[\n]',
 '[unidentified]',
 '[arrowleft]',
 '[arrowright]',
 '[arrowdown]',
 '[capslock]',
 '[control]',
 '[arrowup]',
 '[delete]',
 '[meta]',
 '[f16]',
 '[numlock]',
 '[insert]',
 '[process]',
 '[f11]',
 '[home]',
 '[end]',
 '[tab]',
 '[mediaprevioustrack]',
 '[alt]',
 '[pagedown]',
 '[contextmenu]']

In [19]:
unused_token_slots = [f"[unused{i}]" for i in range(1, len(special_tokens) + 1)]

charbert_token_map = dict(zip(special_tokens, unused_token_slots))
charbert_token_map

{'[shift]': '[unused1]',
 '[backspace]': '[unused2]',
 '[spacebar]': '[unused3]',
 '[enter]': '[unused4]',
 '[\n]': '[unused5]',
 '[unidentified]': '[unused6]',
 '[arrowleft]': '[unused7]',
 '[arrowright]': '[unused8]',
 '[arrowdown]': '[unused9]',
 '[capslock]': '[unused10]',
 '[control]': '[unused11]',
 '[arrowup]': '[unused12]',
 '[delete]': '[unused13]',
 '[meta]': '[unused14]',
 '[f16]': '[unused15]',
 '[numlock]': '[unused16]',
 '[insert]': '[unused17]',
 '[process]': '[unused18]',
 '[f11]': '[unused19]',
 '[home]': '[unused20]',
 '[end]': '[unused21]',
 '[tab]': '[unused22]',
 '[mediaprevioustrack]': '[unused23]',
 '[alt]': '[unused24]',
 '[pagedown]': '[unused25]',
 '[contextmenu]': '[unused26]'}

# Export Special tokens as JSON

In [20]:
#export unused token map
# Convert and write JSON object to file
file_path = os.path.join(data_path, "token_map.json")
with open(file_path, "w") as outfile:
    json.dump(charbert_token_map, outfile)

In [21]:
df['charbert_key'] = df['key_clean'].map(charbert_token_map)
df['charbert_key'] = df['charbert_key'].fillna(df['key_clean'])
df

,key,response_id,response_content,participant_id,sentence_id,sentence_content,diagnosis,keydown,keyup,key_duration_ms,flight_time,key_clean,charbert_key
0,shift,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80077.6719,80672.2719,594.600,433.900,[shift],[unused1]
1,b,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80511.5719,80615.2719,103.700,288.700,b,b
2,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80800.2719,80861.4719,61.200,149.700,o,o
3,o,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,80949.9719,81076.3719,126.400,355.600,o,o
4,s,39,"Books include Penguin Island, a satire on the ...",11,1,"Books include Penguin Island, a satire on the ...",1,81305.5719,81480.7719,175.200,560.900,s,s
...,...,...,...,...,...,...,...,...,...,...,...,...,...
437910,r,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709861.2300,709965.3350,104.105,136.015,r,r
437911,k,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,709997.2450,710077.2900,80.045,168.010,k,k
437912,e,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710165.2550,710309.3200,144.065,248.005,e,e
437913,t,2429,Over three million cattle are residents of the...,1167,15,Over three million cattle are residents of the...,0,710413.2600,710509.4550,96.195,831.980,t,t


In [22]:
sentences = df['sentence_id'].unique()
participants = df['participant_id'].unique()


new_data = []

for i in participants:
    participant_df = df[df['participant_id'] == i]
    diagnosis = participant_df['diagnosis'].unique()[0]

    for j in participant_df['sentence_id'].unique():
        keys = participant_df[participant_df['sentence_id'] == j]['charbert_key'].str.cat(sep=' ')
        timings = participant_df[participant_df['sentence_id'] == j]['key_duration_ms'].to_list()
        flights = participant_df[participant_df['sentence_id'] == j]['flight_time'].to_list()
        # keys = str(keys)
        response_id = participant_df[(participant_df['sentence_id'] == j)]['response_id'].unique()
        response_content = participant_df[(participant_df['sentence_id'] == j)]['response_content'].unique()
        sentence_content = participant_df[(participant_df['sentence_id'] == j)]['sentence_content'].unique()

        if len(keys) != 0 and len(response_id) != 0 and len(response_content) != 0 and len(sentence_content) != 0:
            new_data.append({
                'participant_id': i,
                'sentence_id': j,
                'key_sequence': keys,
                'timing_sequence': timings,
                'flight_time': flights,
                'diagnosis': diagnosis,
                'response_id': response_id[0],
                'response_content': response_content[0],
                'sentence_content': sentence_content[0]
            })
        else:
            print(f"Missing data for participant {i}, sentence {j}")

new_df = pd.DataFrame(new_data)
new_df

,participant_id,sentence_id,key_sequence,timing_sequence,flight_time,diagnosis,response_id,response_content,sentence_content
0,11,1,[unused1] b o o s [unused2] k s s [unused3] i ...,"[594.6000000000058, 103.70000000001164, 61.199...","[433.8999999999942, 288.70000000001164, 149.69...",1,39,"Books include Penguin Island, a satire on the ...","Books include Penguin Island, a satire on the ..."
1,11,2,"[unused1] h o w e v e r , [unused3] r e l i g ...","[328.5, 31.600000000002183, 76.79999999999927,...","[278.5, 692.1000000000022, 150.5, 269.70000000...",1,37,"However, religions other than Islam, use a dif...","However, religions other than Islam, use a dif..."
2,11,3,[unused1] t h e [unused3] [unused1] f r a n k ...,"[448.3999999999796, 114.30000000001746, 97.200...","[287.49999999998545, 252.60000000000582, 112.3...",1,41,The Franks alliance was important exactly beca...,The Franks alliance was important exactly beca...
3,11,4,[unused1] h s [unused2] e [unused3] i s [unuse...,"[255.8000000000029, 140.29999999999563, 65.0, ...","[199.60000000000582, 433.79999999999563, 601.5...",1,38,"He is buried in Egypt, Aswan at the Mausoleum...","He is buried in Egypt, Aswan at the Mausoleum ..."
4,11,5,[unused1] t h e [unused3] w - s h a p e d [unu...,"[470.3000000000029, 145.60000000000582, 126.69...","[410.1999999999971, 285.1000000000058, 145.199...",1,40,The w-shaped glpyh above the second consonant ...,The w-shaped glyph above the second consonant ...
...,...,...,...,...,...,...,...,...,...
3429,1167,11,[unused1] t h e [unused3] n o v e l [unused3] ...,"[528.0299999279669, 80.06000006600516, 56.0749...","[359.95999991399003, 240.00500002899207, 120.0...",0,2422,The novel explores the relationship between P...,The novel explores the relationship between Pa...
3430,1167,12,[unused1] s p l i t - f i n g e r [unused3] a ...,"[568.1600000238977, 160.4300000470248, 71.9849...","[407.95999998186016, 208.0850000489154, 199.90...",0,2424,Split-finger aiming requires the archer to pla...,Split-finger aiming requires the archer to pla...
3431,1167,13,[unused1] t h e y [unused3] f o u g h t [unuse...,"[392.04499998595566, 72.10500002803747, 88.035...","[207.9449999149656, 232.034999993979, 136.0100...",0,2427,They fought a thirty years war on the side of ...,They fought a thirty years war on the side of ...
3432,1167,14,"[unused1] h o w e v e r , [unused3] t h e r e ...","[576.0049999230541, 88.08000001590699, 72.1200...","[343.9199998980621, 320.0100000250386, 239.980...",0,2428,"However, there is no evidence that those tatto...","However, there is no evidence that those tatto..."


In [23]:
#filter out response_content that is NaN
filt_df = new_df.loc[~new_df['response_content'].isnull()]
filt_df

,participant_id,sentence_id,key_sequence,timing_sequence,flight_time,diagnosis,response_id,response_content,sentence_content
0,11,1,[unused1] b o o s [unused2] k s s [unused3] i ...,"[594.6000000000058, 103.70000000001164, 61.199...","[433.8999999999942, 288.70000000001164, 149.69...",1,39,"Books include Penguin Island, a satire on the ...","Books include Penguin Island, a satire on the ..."
1,11,2,"[unused1] h o w e v e r , [unused3] r e l i g ...","[328.5, 31.600000000002183, 76.79999999999927,...","[278.5, 692.1000000000022, 150.5, 269.70000000...",1,37,"However, religions other than Islam, use a dif...","However, religions other than Islam, use a dif..."
2,11,3,[unused1] t h e [unused3] [unused1] f r a n k ...,"[448.3999999999796, 114.30000000001746, 97.200...","[287.49999999998545, 252.60000000000582, 112.3...",1,41,The Franks alliance was important exactly beca...,The Franks alliance was important exactly beca...
3,11,4,[unused1] h s [unused2] e [unused3] i s [unuse...,"[255.8000000000029, 140.29999999999563, 65.0, ...","[199.60000000000582, 433.79999999999563, 601.5...",1,38,"He is buried in Egypt, Aswan at the Mausoleum...","He is buried in Egypt, Aswan at the Mausoleum ..."
4,11,5,[unused1] t h e [unused3] w - s h a p e d [unu...,"[470.3000000000029, 145.60000000000582, 126.69...","[410.1999999999971, 285.1000000000058, 145.199...",1,40,The w-shaped glpyh above the second consonant ...,The w-shaped glyph above the second consonant ...
...,...,...,...,...,...,...,...,...,...
3429,1167,11,[unused1] t h e [unused3] n o v e l [unused3] ...,"[528.0299999279669, 80.06000006600516, 56.0749...","[359.95999991399003, 240.00500002899207, 120.0...",0,2422,The novel explores the relationship between P...,The novel explores the relationship between Pa...
3430,1167,12,[unused1] s p l i t - f i n g e r [unused3] a ...,"[568.1600000238977, 160.4300000470248, 71.9849...","[407.95999998186016, 208.0850000489154, 199.90...",0,2424,Split-finger aiming requires the archer to pla...,Split-finger aiming requires the archer to pla...
3431,1167,13,[unused1] t h e y [unused3] f o u g h t [unuse...,"[392.04499998595566, 72.10500002803747, 88.035...","[207.9449999149656, 232.034999993979, 136.0100...",0,2427,They fought a thirty years war on the side of ...,They fought a thirty years war on the side of ...
3432,1167,14,"[unused1] h o w e v e r , [unused3] t h e r e ...","[576.0049999230541, 88.08000001590699, 72.1200...","[343.9199998980621, 320.0100000250386, 239.980...",0,2428,"However, there is no evidence that those tatto...","However, there is no evidence that those tatto..."


# Export Cleaned Data

In [24]:
filt_df.to_csv(os.path.join(data_path, 'cleaned_data_with_ft.csv'), index=False)